In [1]:
! pip install pinecone-client langchain openai wikipedia google-api-python-client unstructured tabulate pdf2image python-dotenv tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 9.

In [2]:
from google.colab import drive

drive.mount('/content/drive')
directory_path = '/content/drive/MyDrive/Dataset/'

Mounted at /content/drive


In [3]:
from langchain.document_loaders import DirectoryLoader

docmnts = DirectoryLoader(directory_path, glob='**/*.pdf', show_progress=True).load()

  0%|          | 0/6 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 6/6 [00:23<00:00,  3.86s/it]


In [4]:
from langchain.text_splitter import CharacterTextSplitter

split_documnts = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0).split_documents(docmnts)

In [60]:
# import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai_api_key = os.environ['OPENAI_API_KEY']
pinecone_env = os.environ['PINECONE_ENV']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [7]:
import pinecone
import tiktoken 
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings

# we use the openAI embedding model
embeddings = OpenAIEmbeddings()
pinecone.init(api_key=pinecone_api_key, environment=pinecone_env)
db_dcmnt = Pinecone.from_documents(split_documnts, embeddings, index_name='document-indexing')

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
query = "What are the important nutrients?"
search_docs = db_dcmnt.similarity_search(query)
search_docs

[Document(page_content='BASIC NUTRITION 101\n\nTo begin with, an understanding of the basic fundamentals of food and nutrition is essential for anyone who wants to maintain a healthful lifestyle or to remedy some bad habits that have resulted in the necessity to make lifestyle changes for health and for a more rewarding life overall. As we age, the body naturally ages along with us and systems that once worked so well when we were younger may encounter difficulties sustaining their vital functions at or near previous levels. Many of these things that affect the efficiency of our bodily functions are caused by us and the 2 biggest culprits are nutrition and fitness. The following is a very surface explanation of some of the most basic concepts of nutrition.\n\nMarco nutrients:\n\nCarbohydrates: There are 2 types of carbohydrates and 3 if you include fiber;\n\nhowever, since fiber contributes no actual nutrients, it is included for personal understanding. Protein: Fats: Saturated/ unsatu

In [10]:
from langchain import OpenAI, LLMChain
from langchain.chains import RetrievalQA

llm = OpenAI()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=db_dcmnt.as_retriever(),)

query = "Which foods contain vitamin-a?"
result = qa.run(query)

result

' Fruits and vegetables that contain vitamin A include carrots, sweet potatoes, spinach, melons, oranges, orange juice, plums, raspberries, strawberries, and tangerines.'

In [50]:
from re import VERBOSE
from langchain import OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
# adding an agent 
from langchain.agents import load_tools, initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory

llm = OpenAI()
memory = ConversationBufferMemory(memory_key='chat_history')

tools = load_tools(['wikipedia'], llm=llm)

agent = initialize_agent( tools, llm, agent= AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose= True, memory= memory)
agent.run('What is are some of the high-fiber foods?')



> Entering new AgentExecutor chain...
 I need to research what the high-fiber foods are.
Action: Wikipedia
Action Input: High-fiber foods
Observation: Page: Dietary fiber
Summary: Dietary fiber (in Commonwealth English fibre) or roughage is the portion of plant-derived food that cannot be completely broken down by human digestive enzymes. Dietary fibers are diverse in chemical composition, and can be grouped generally by their solubility, viscosity, and fermentability, which affect how fibers are processed in the body. Dietary fiber has two main components: soluble fiber and insoluble fiber, which are components of plant-based foods, such as legumes, whole grains and cereals, vegetables, fruits, and nuts or seeds. A diet high in regular fiber consumption is generally associated with supporting health and lowering the risk of several diseases. Dietary fiber consists of non-starch polysaccharides and other plant components such as cellulose, resistant starch, resistant dextrins, inulin

'The best high-fiber foods include legumes, whole grains and cereals, vegetables, fruits, and nuts or seeds. Additionally, dietary fibers such as beta-glucans, resistant starch, xanthan gum, and inulin are beneficial for gastrointestinal health.'

In [51]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key='chat_history')

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
    verbose=True,
    memory=memory
)

In [42]:
agent.run('how much fiber should a healthy adult need?')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: A healthy adult should aim to consume at least 25 to 30 grams of fiber per day. This can come from a variety of sources, including fruits, vegetables, whole grains, beans, and nuts. Adequate fiber intake can help regulate digestion, lower cholesterol levels, and reduce the risk of certain diseases.

> Finished chain.


'A healthy adult should aim to consume at least 25 to 30 grams of fiber per day. This can come from a variety of sources, including fruits, vegetables, whole grains, beans, and nuts. Adequate fiber intake can help regulate digestion, lower cholesterol levels, and reduce the risk of certain diseases.'

In [43]:
agent.run('how do I get so much fiber into my diet?')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: There are many ways to increase your fiber intake. Here are some tips:
- Start your day with a high-fiber breakfast cereal or oatmeal.
- Snack on fruits and vegetables throughout the day.
- Choose whole grain breads, pasta, and rice instead of their refined counterparts.
- Add beans or lentils to soups, stews, and salads.
- Incorporate nuts and seeds into your diet, such as almond butter on toast or chia seeds in a smoothie.
By making small changes to your diet, you can gradually increase your fiber intake and enjoy the many health benefits that come with it.

> Finished chain.


'There are many ways to increase your fiber intake. Here are some tips:\n- Start your day with a high-fiber breakfast cereal or oatmeal.\n- Snack on fruits and vegetables throughout the day.\n- Choose whole grain breads, pasta, and rice instead of their refined counterparts.\n- Add beans or lentils to soups, stews, and salads.\n- Incorporate nuts and seeds into your diet, such as almond butter on toast or chia seeds in a smoothie.\nBy making small changes to your diet, you can gradually increase your fiber intake and enjoy the many health benefits that come with it.'

In [52]:
from langchain.agents import Tool

name = "Food and Nutrition Database"

description= """
                useful for when you need to answer questions about macronutrients, micronutrients, digestion, 
                energy metabolism, dietary guidelines, meal planning, portion control, food storage, cooking techniques,
                balanced nutrition, food labels, the five food groups, plant-based diets, and their benefits """

search_tool = Tool(
    name=name,
    func=qa.run,
    description=description,
)

tools.append(search_tool)

In [53]:
from langchain.experimental import plan_and_execute
from langchain.experimental.plan_and_execute import (PlanAndExecute, load_agent_executor, load_chat_planner)

memory = ConversationBufferMemory(memory_key= 'chat_history')
planner = load_chat_planner(llm)
executor = load_agent_executor(llm, tools, verbose= True)

agent = PlanAndExecute(planner= planner, executor= executor, verbose= True, reduce_k_below_max_tokens= True)


In [55]:
agent.run('Is Vitamin-k found in carrots?')



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the nutrients found in carrots '), Step(value='Determine whether Vitamin-K is found in carrots '), Step(value="Given the above steps taken, please respond to the user's original question \n")]

> Entering new AgentExecutor chain...

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Nutrition in Carrots"
}
```

Observation: Page: Carrot
Summary: The carrot (Daucus carota subsp. sativus) is a root vegetable, typically orange in color, though purple, black, red, white, and yellow cultivars exist, all of which are domesticated forms of the wild carrot, Daucus carota, native to Europe and Southwestern Asia. The plant probably originated in Persia and was originally cultivated for its leaves and seeds. The most commonly eaten part of the plant is the taproot, although the stems and leaves are also eaten. The domestic carrot has been selectively bred for its enlarged, more palatable, less woody-textured taproot.
The 

'Carrots contain Vitamins A and C, folate, iron, magnesium, and Vitamin K. A serving size is 1/2 cup cooked or a small glass of 100% vegetable juice.'

In [59]:
agent.run('I have aneamia so how do I increase my iron consumption?')



> Entering new PlanAndExecute chain...
steps=[Step(value='Consult a physician to determine the cause of aneamia'), Step(value='Get a blood test to confirm diagnosis and determine the severity of aneamia'), Step(value='Based on the severity of the aneamia, determine the appropriate treatment plan to increase iron levels'), Step(value='Follow the treatment plan provided by the physician'), Step(value='Monitor iron levels to ensure they stay within healthy range'), Step(value="Given the above steps taken, please respond to the user's original question by recommending dietary sources of iron as well as iron supplements if prescribed by the physician.\n")]

> Entering new AgentExecutor chain...

Action:
```
{
  "action": "Wikipedia",
  "action_input": "Anaemia"
}
```

Observation: Page: Anemia
Summary: Anemia or anaemia (British English) is a blood disorder in which the blood has a reduced ability to carry oxygen due to a lower than normal number of red blood cells, or a reduction in the 

'It is important to consult a doctor or nutritionist for a personalized plan, as dietary sources of iron and iron supplements may be recommended depending on the severity of the anemia. Foods high in iron include meat, nuts, and foods made with iron-fortified flour.'